In [1]:
import sys
sys.path.append("../..")
import dill as pickle
import copy

import nnscaler.ir.adapter.prim as PTypes
from nnscaler.codegen.module.module import ModuleCodeGen
from nnscaler.ir.adapter import IRWeightReducer, IRAdapter
from nnscaler.ir.operator import IRDataOperation, IRBpOperation, IRFwOperation
from nnscaler.ir.adapter.prim import ChunkPrim, IRAdapterPrim
from nnscaler.graph.graph import IRSegment
from nnscaler.graph.function.dimops import IRDimops
from nnscaler.codegen.module.module import ModuleCodeGen
from nnscaler.execplan.execplan import ExeReuseCell, IRCell
from nnscaler.ir.tensor import IRSubTensor, IRFullTensor

skip apex ops as it is not installed.


Load baseline model.

In [2]:
mg_path = "../gen_model/mgeners/llama3_default_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
with open(mg_path, "rb") as fp:
    mg_org = pickle.load(fp)

/home/azureuser/miniconda3/envs/verdict/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [3]:
for x in mg_org.execplan._seq[0]:
    print(x)

ReuseCell-(0,)(name=dataloader1487, inputs=(Object(dataloader8752, val=None, is_constant=True),), outputs=(t8751(p8750,(8, 8192),d(),v(0/1)),))
ReuseCell-[0](name=segment2947, inputs=(t8751(p8750,(8, 8192),d(),v(0/1)),), outputs=(t6945(p6,(1, 1, 8192, 8192),d(),v(0/1)), t7841(p1925,(8, 8192, 128),d(),v(0/1)), t7842(p1927,(8, 8192, 128),d(),v(0/1)), t7846(p1935,(8, 8192, 1),d(),v(0/1))))
ReuseCell-(0,)(name=adapter5085, inputs=(t7841(p1925,(8, 8192, 128),d(),v(0/1)),), outputs=())
ReuseCell-(0,)(name=adapter5127, inputs=(t6945(p6,(1, 1, 8192, 8192),d(),v(0/1)),), outputs=())
ReuseCell-(0,)(name=adapter5146, inputs=(t7842(p1927,(8, 8192, 128),d(),v(0/1)),), outputs=())
ReuseCell-(0,)(name=adapter5198, inputs=(t7846(p1935,(8, 8192, 1),d(),v(0/1)),), outputs=())
ReuseCell-(0,)(name=dataloader1487, inputs=(Object(dataloader8752, val=None, is_constant=True),), outputs=(t23721(p23720,(8, 8192),d(),v(0/1)),))
ReuseCell-[0](name=segment2947, inputs=(t23721(p23720,(8, 8192),d(),v(0/1)),), output

Display nodes.

In [4]:
def traverse(seq, func):
    for x in seq:
        if type(x) is ExeReuseCell:
            x = x.cell
        nodes = []
        if type(x) is IRSegment:
            nodes = x.nodes()
        else:
            nodes = [x]
        for node in nodes:
            func(node)

def dump(path, mg):
    with open(path, "wb") as fp:
        pickle.dump(mg, fp)
        
def print_node(node):
    print(node)
    if type(node) is IRAdapter:
        for prim in node.prims:
            print("    ", prim)
traverse(mg_org.execplan._seq[2], print_node)

Adapter-5085(2,)(inputs=(), outputs=(t7841(p1925,(8, 8192, 128),d(),v(0/1)),))
     [t12699(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))] = move[2]([], src=0, dst=2)
     [t7841(p1925,(8, 8192, np.int64(128)),d(),v(0/1))] = all_gather[[2]]([t12699(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))])
Adapter-5127(2,)(inputs=(), outputs=(t6945(p6,(1, 1, 8192, 8192),d(),v(0/1)),))
     [t12711(p6,(1, 1, 8192, np.int64(4096)),d(3,),v(0/1))] = move[2]([], src=0, dst=2)
     [t6945(p6,(1, 1, 8192, np.int64(8192)),d(),v(0/1))] = all_gather[[2]]([t12711(p6,(1, 1, 8192, np.int64(4096)),d(3,),v(0/1))])
Adapter-5146(2,)(inputs=(), outputs=(t7842(p1927,(8, 8192, 128),d(),v(0/1)),))
     [t12721(p1927,(8, 8192, np.int64(64)),d(2,),v(0/1))] = move[2]([], src=0, dst=2)
     [t7842(p1927,(8, 8192, np.int64(128)),d(),v(0/1))] = all_gather[[2]]([t12721(p1927,(8, 8192, np.int64(64)),d(2,),v(0/1))])
Adapter-5198(2,)(inputs=(), outputs=(t7846(p1935,(8, 8192, 1),d(),v(0/1)),))
     [t12733(p1935,(8, 4096, np.int64(

BR1

In [45]:
# missing all reduce (replaced by identity)
mg_path_new = "../gen_model/mgeners/llama3_br1_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

def mutate(node):
    if type(node) is IRAdapter and node.cid == 5991:
        allreduce_prim = node.prims[0]
        new_prim = PTypes.IdentityAllreducePrim(allreduce_prim._inputs, allreduce_prim._outputs)
        node._prims[0] = new_prim
        print(node.prims[0], node._inputs, node._outputs)

traverse(mg.execplan._seq[0], mutate)
traverse(mg.execplan._seq[1], mutate)
dump(mg_path_new, mg)

[t7000(p123,(8, 8192, np.int64(128)),d(),v(0/1))] = identity_allreduce[[0]]([t11013(p123,(8, 8192, 128),d(),v(0/2))]) [t11013(p123,(8, 8192, 128),d(),v(0/2))] [t7000(p123,(8, 8192, 128),d(),v(0/1))]
[t7000(p123,(8, 8192, np.int64(128)),d(),v(0/1))] = identity_allreduce[[0]]([t11013(p123,(8, 8192, 128),d(),v(0/2))]) [t11013(p123,(8, 8192, 128),d(),v(0/2))] [t7000(p123,(8, 8192, 128),d(),v(0/1))]
[t7000(p123,(8, 8192, np.int64(128)),d(),v(0/1))] = identity_allreduce[[1]]([t11014(p123,(8, 8192, 128),d(),v(1/2))]) [t11014(p123,(8, 8192, 128),d(),v(1/2))] [t7000(p123,(8, 8192, 128),d(),v(0/1))]
[t7000(p123,(8, 8192, np.int64(128)),d(),v(0/1))] = identity_allreduce[[1]]([t11014(p123,(8, 8192, 128),d(),v(1/2))]) [t11014(p123,(8, 8192, 128),d(),v(1/2))] [t7000(p123,(8, 8192, 128),d(),v(0/1))]


BR2

In [46]:
# missing all reduce (drop the prim)
mg_path_new = "../gen_model/mgeners/llama3_br2_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

def mutate(node):
    if type(node) is IRAdapter and node.cid == 5991:
        node._prims = []
        print(node.prims)

traverse(mg.execplan._seq[0], mutate)
traverse(mg.execplan._seq[1], mutate)
dump(mg_path_new, mg)

[]
[]
[]
[]


BR3

In [47]:
# wrong prim replacement (allreduce -> allgather)
mg_path_new = "../gen_model/mgeners/llama3_br3_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

def mutate(node):
    if type(node) is IRAdapter and node.cid == 5991:
        allreduce_prim = node.prims[0]
        new_prim = PTypes.AllGatherPrim(allreduce_prim._inputs, allreduce_prim._outputs, 0)
        node._prims[0] = new_prim
        print(node.prims[0], node._inputs, node._outputs)

traverse(mg.execplan._seq[0], mutate)
traverse(mg.execplan._seq[1], mutate)
dump(mg_path_new, mg)

[t7000(p123,(8, 8192, np.int64(128)),d(),v(0/1))] = all_gather[[0]]([t11013(p123,(8, 8192, 128),d(),v(0/2))]) [t11013(p123,(8, 8192, 128),d(),v(0/2))] [t7000(p123,(8, 8192, 128),d(),v(0/1))]
[t7000(p123,(8, 8192, np.int64(128)),d(),v(0/1))] = all_gather[[0]]([t11013(p123,(8, 8192, 128),d(),v(0/2))]) [t11013(p123,(8, 8192, 128),d(),v(0/2))] [t7000(p123,(8, 8192, 128),d(),v(0/1))]
[t7000(p123,(8, 8192, np.int64(128)),d(),v(0/1))] = all_gather[[1]]([t11014(p123,(8, 8192, 128),d(),v(1/2))]) [t11014(p123,(8, 8192, 128),d(),v(1/2))] [t7000(p123,(8, 8192, 128),d(),v(0/1))]
[t7000(p123,(8, 8192, np.int64(128)),d(),v(0/1))] = all_gather[[1]]([t11014(p123,(8, 8192, 128),d(),v(1/2))]) [t11014(p123,(8, 8192, 128),d(),v(1/2))] [t7000(p123,(8, 8192, 128),d(),v(0/1))]


BR4

In [6]:
# wrong prim replacement (allreduce -> allgather)
mg_path_new = "../gen_model/mgeners/llama3_br4_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

def mutate(node):
    if type(node) is IRAdapter and node.cid == 5085:
        allgather_prim = node.prims[1]
        new_prim = PTypes.AllReducePrim(allgather_prim._inputs, allgather_prim._outputs)
        node._prims[1] = new_prim
        print(node.prims[1], node._inputs, node._outputs)

traverse(mg.execplan._seq[2], mutate)
traverse(mg.execplan._seq[3], mutate)
dump(mg_path_new, mg)

[t7841(p1925,(8, 8192, np.int64(128)),d(),v(0/1))] = all_reduce[[2]]([t12699(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))]) [] [t7841(p1925,(8, 8192, 128),d(),v(0/1))]
[t7841(p1925,(8, 8192, np.int64(128)),d(),v(0/1))] = all_reduce[[2]]([t12699(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))]) [] [t7841(p1925,(8, 8192, 128),d(),v(0/1))]
[t7841(p1925,(8, 8192, np.int64(128)),d(),v(0/1))] = all_reduce[[3]]([t12700(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))]) [] [t7841(p1925,(8, 8192, 128),d(),v(0/1))]
[t7841(p1925,(8, 8192, np.int64(128)),d(),v(0/1))] = all_reduce[[3]]([t12700(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))]) [] [t7841(p1925,(8, 8192, 128),d(),v(0/1))]


BR5

In [49]:
# bw missing all reduce (replaced by identity)
mg_path_new = "../gen_model/mgeners/llama3_br5_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

def mutate(node):
    if type(node) is IRAdapter and node.cid == 12736:
        allreduce_prim = node.prims[0]
        new_prim = PTypes.AllReduceIdentityPrim(allreduce_prim._inputs, allreduce_prim._outputs)
        node._prims[0] = new_prim
        print(node.prims[0], node._inputs, node._outputs)

traverse(mg.execplan._seq[2], mutate)
traverse(mg.execplan._seq[3], mutate)
dump(mg_path_new, mg)

[t8000(p2267,(8, 8192, 128),d(),v(0/1))] = allreduce_identity[[2]]([t8000(p2267,(8, 8192, 128),d(),v(0/1))]) [t8000(p2267,(8, 8192, 128),d(),v(0/1))] [t8000(p2267,(8, 8192, 128),d(),v(0/1))]
[t8000(p2267,(8, 8192, 128),d(),v(0/1))] = allreduce_identity[[2]]([t8000(p2267,(8, 8192, 128),d(),v(0/1))]) [t8000(p2267,(8, 8192, 128),d(),v(0/1))] [t8000(p2267,(8, 8192, 128),d(),v(0/1))]
[t8000(p2267,(8, 8192, 128),d(),v(0/1))] = allreduce_identity[[3]]([t8000(p2267,(8, 8192, 128),d(),v(0/1))]) [t8000(p2267,(8, 8192, 128),d(),v(0/1))] [t8000(p2267,(8, 8192, 128),d(),v(0/1))]
[t8000(p2267,(8, 8192, 128),d(),v(0/1))] = allreduce_identity[[3]]([t8000(p2267,(8, 8192, 128),d(),v(0/1))]) [t8000(p2267,(8, 8192, 128),d(),v(0/1))] [t8000(p2267,(8, 8192, 128),d(),v(0/1))]


BR6

In [50]:
# rank
mg_path_new = "../gen_model/mgeners/llama3_br6_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

def mutate(node):
    if type(node) is IRAdapter and node.cid == 5085 and node.device == (0,):
        move_prim = node.prims[1]
        move_prim.kwargs["dst"] = 3
        print(move_prim, move_prim.kwargs)
    if type(node) is IRAdapter and node.cid == 5085 and node.device == (1,):
        move_prim = node.prims[1]
        move_prim.kwargs["dst"] = 2
        print(move_prim, move_prim.kwargs)
    if type(node) is IRAdapter and node.cid == 5085 and node.device == (2,):
        move_prim = node.prims[0]
        move_prim.kwargs["src"] = 1
        print(move_prim, move_prim.kwargs)
    if type(node) is IRAdapter and node.cid == 5085 and node.device == (3,):
        move_prim = node.prims[0]
        move_prim.kwargs["src"] = 0
        print(move_prim, move_prim.kwargs)

traverse(mg.execplan._seq[0], mutate)
traverse(mg.execplan._seq[1], mutate)
traverse(mg.execplan._seq[2], mutate)
traverse(mg.execplan._seq[3], mutate)
dump(mg_path_new, mg)

[] = move[0]([t12699(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))], src=0, dst=3) {'shape': (8, 8192, np.int64(64)), 'dtype': 'torch.float32', 'src': 0, 'dst': 3}
[] = move[0]([t12699(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))], src=0, dst=3) {'shape': (8, 8192, np.int64(64)), 'dtype': 'torch.float32', 'src': 0, 'dst': 3}
[] = move[1]([t12700(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))], src=1, dst=2) {'shape': (8, 8192, np.int64(64)), 'dtype': 'torch.float32', 'src': 1, 'dst': 2}
[] = move[1]([t12700(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))], src=1, dst=2) {'shape': (8, 8192, np.int64(64)), 'dtype': 'torch.float32', 'src': 1, 'dst': 2}
[t12699(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))] = move[2]([], src=1, dst=2) {'shape': (8, 8192, np.int64(64)), 'dtype': 'torch.float32', 'src': 1, 'dst': 2}
[t12699(p1925,(8, 8192, np.int64(64)),d(2,),v(0/1))] = move[2]([], src=1, dst=2) {'shape': (8, 8192, np.int64(64)), 'dtype': 'torch.float32', 'src': 1, 'dst': 2}
[t12700(p1925,(8, 8192, np.i

BR7

In [51]:
# single-rank mutant on fw ADD op inputs const
mg_path_new = "../gen_model/mgeners/llama3_br7_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

def mutate(node):
    if type(node) is IRDimops and node.name == "add" and node.cid == 6:
        node._inputs[1] = 2
        print(node._inputs)

traverse(mg.execplan._seq[0], mutate)
dump(mg_path_new, mg)

[t6948(p11,(8, 8192, 1),d(),v(0/1)), 2]
[t6948(p11,(8, 8192, 1),d(),v(0/1)), 2]


BR8

In [52]:
# single-rank mutant on fw POW op inputs const
mg_path_new = "../gen_model/mgeners/llama3_br8_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

def mutate(node):
    if type(node) is IRDimops and node.name == "pow" and node.cid == 36:
        node._inputs[1] = 1
        print(node._inputs)

traverse(mg.execplan._seq[0], mutate)
dump(mg_path_new, mg)

[t12863(p12861,(8, 8192, 128),d(),v(0/1)), 1]
[t12863(p12861,(8, 8192, 128),d(),v(0/1)), 1]


BR9

In [53]:
# multi-rank mutant on fw ADD op input const
mg_path_new = "../gen_model/mgeners/llama3_br9_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

def mutate(node):
    if type(node) is IRDimops and node.name == "add" and node.cid == 742:
        node._inputs[1] = 2
        print(node._inputs)

traverse(mg.execplan._seq[0], mutate)
traverse(mg.execplan._seq[1], mutate)
dump(mg_path_new, mg)

[t7844(p1931,(8, 8192, 1),d(),v(0/1)), 2]
[t7844(p1931,(8, 8192, 1),d(),v(0/1)), 2]
[t7844(p1931,(8, 8192, 1),d(),v(0/1)), 2]
[t7844(p1931,(8, 8192, 1),d(),v(0/1)), 2]


BR10

In [54]:
# single-rank mutant on bw MULTIREF op input tensor
mg_path_new = "../gen_model/mgeners/llama3_br10_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

mutated = False
def mutate(node):
    global mutated
    if type(node) is IRBpOperation and node.mirror.name == "multiref" and node.cid == 12021:
        if not mutated:
            node._inputs = node._inputs + node._inputs[:1]
            print(node._inputs)
            mutated = True
            

traverse(mg.execplan._seq[2], mutate)
dump(mg_path_new, mg)

[g20577(p20575,(8, 8192, 128),d(),v(0/1)), g20581(p20579,(8, 8192, 128),d(),v(0/1)), g20585(p20583,(8, 8192, 128),d(),v(0/1)), g20577(p20575,(8, 8192, 128),d(),v(0/1))]


BR11

In [55]:
# single-rank mutant on bw APPLY_ROTARY_EMB op input tensor
mg_path_new = "../gen_model/mgeners/llama3_br11_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

mutated = False
def mutate(node):
    global mutated
    if type(node) is IRBpOperation and node.mirror.name == "apply_rotary_emb" and node.cid == 4392:
        if not mutated:
            node._inputs = node._inputs[::-1]
            print(node._inputs)
            mutated = True
            

traverse(mg.execplan._seq[2], mutate)
dump(mg_path_new, mg)

[g9869(p1972,(8, 8192, 32, 4),d(),v(0/1)), g9868(p1970,(8, 8192, 32, 4),d(),v(0/1))]


BR12

In [56]:
# single-rank mutant on div op scaling const
mg_path_new = "../gen_model/mgeners/llama3_br12_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

stale_tensor = None
def mutate(node):
    global stale_tensor
    if type(node) is IRDimops and node.name == "div" and node.cid == 116:
        node._inputs[1] = 4
        print(node._inputs)

traverse(mg.execplan._seq[0], mutate)
dump(mg_path_new, mg)

[t7083(p305,(8, 32, 8192, 8192),d(),v(0/1)), 4]
[t7083(p305,(8, 32, 8192, 8192),d(),v(0/1)), 4]


BR13

In [57]:
# multi-rank mutant on div op scaling const
mg_path_new = "../gen_model/mgeners/llama3_br13_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

stale_tensor = None
def mutate(node):
    global stale_tensor
    if type(node) is IRDimops and node.name == "div" and node.cid == 116:
        node._inputs[1] = 4
        print(node._inputs)

traverse(mg.execplan._seq[0], mutate)
traverse(mg.execplan._seq[1], mutate)
dump(mg_path_new, mg)

[t7083(p305,(8, 32, 8192, 8192),d(),v(0/1)), 4]
[t7083(p305,(8, 32, 8192, 8192),d(),v(0/1)), 4]
[t7083(p305,(8, 32, 8192, 8192),d(),v(0/1)), 4]
[t7083(p305,(8, 32, 8192, 8192),d(),v(0/1)), 4]


BR14

In [58]:
# missing mb=2 last op
mg_path_new = "../gen_model/mgeners/llama3_br14_dp2_pp2_tp2_nm2_gbs32_ly32_h32_hi128_sq8192.pkl"
mg = copy.deepcopy(mg_org)

mg.execplan._seq[0] = mg.execplan._seq[0][:-1]
dump(mg_path_new, mg)